# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

Overall Goals
- Understand what factors make a car more or less expensive. 
- Provide clear recommendations to your client/dealership as to what consumers value in a used car.
- Basic report that details your primary findings. Your audience for this report is a group of used car dealers interested in fine-tuning their inventory.

- Organized folder on Git with README 
- Syntax and Comments
- Visualizations
- Modeling 
    - multiple regression models, CV of models, Grid Search hyperparams, 
    - interpret coeffs, eval metrics and rational for use of metric
- Findings 
    - Clearly stated bus understanding
    - Clean and organized
    - Correct and concise interpretation
    - own section of findings with actionable items highlighted for a non-technical audience
    - Next steps 

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

#### Business Question
At its core, the business question at hand is "how can a car dearler fine tune his inventory to optimize for sales price?" Or stated differently, what are the key drivers that will cause customers to want to pay more for the cars? 

#### Data Problem Definition
From a data mining perspective, our goal is to understand which features impact pricing the most. Are there any clear correlations between some features and pricing?  Is there a cluster of cars that tend to sell for more?  Can we identify a number of Principle Components that drive the price? Can we find a model that will predict the car price with an error rate of less than 10%?  How does all this translate into clear recommendations for the car dealership to fine tune his inventory? 

#### Constraints and Assumptions:
- We do note that this question only looks at price and gives no consideration to the dealer's profit marin. 
- Since the dataset includes cars from all over the country, we will assume this is a natiowide dealership

#### Inventory and Resources
 - This is a project due within less than a week with only one data scientist with limited experience working on it 

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
#Get familiar with the data, 
#identify data quality problems, 
#discover first insights into the data, 
#detect interesting subsets to form hypotheses for hidden information

In this section we will: 
- Load and preview the data
- Understand and describe the features
- Get basic information about the dataset size and data types
- Quick glance at numeric values, min, max, means and standard deviation
- Look for duplicate rows
- Inspect blanks, nulls, null ratios, 
- Identify any outliers or artifical ceilings
- Identify values and counts for each column

- First insights
- Detection interesting subsets of data
- Any points that can help us inform our business understanding

#### Understand and describe the features
 - id: Unique ID for car
 - region: Area car is parked
 - price:  Price of the vehicle
 - year: Manufacturing Year
 - manufacturer: Manufacturing Company
 - model: Car model
 - condition: Car condition
 - cylinders: Number of cylinders in the car
 - fuel: Type of fuel the car consumes
 - odometer: mileage reading from the odometer
 - title_status: Is the title clean - or are there any legal issues with it?
 - transmission: transmission type
 - VIN: Vehicle Identification Number
 - drive: Drivetrain type
 - size: ? 
 - type: vehicle form factor
 - paint_color:  Paint Color
 - state: state where vehicle is located

In [6]:
import numpy as np
import pandas as pd
from IPython.display import display

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'

from scipy.linalg import svd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_transformer, make_column_selector, TransformedTargetRegressor
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel, RFE
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso, HuberRegressor, Ridge
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error

from sklearn import set_config
set_config(display="diagram")

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.datasets import nile
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.graphics.tsaplots as tsaplots
from statsmodels.tsa.stattools import acf

from statsmodels.tsa.filters.filtertools import convolution_filter
from statsmodels.tsa.seasonal import _extrapolate_trend
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.tsa.forecasting.stl import STLForecast
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa import arima_process
from statsmodels.tsa.stattools import adfuller

#### Load and preview the data

In [31]:
raw = pd.read_csv('data/vehicles.csv')
pd.set_option('display.max_columns', None)
raw.sample(3)

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
328471,7312891496,erie,7988,2011.0,ford,fusion se,excellent,4 cylinders,gas,104750.0,clean,automatic,NaN,fwd,mid-size,sedan,red,pa
164081,7303920434,waterloo / cedar falls,27995,2017.0,bmw,5-series,good,4 cylinders,gas,50194.0,clean,automatic,WBAJA7C34HG905907,4wd,mid-size,sedan,blue,ia
3749,7302204738,huntsville / decatur,24590,2017.0,dodge,challenger sxt coupe 2d,good,6 cylinders,gas,31936.0,clean,automatic,2C3CDZAG7HH665569,rwd,NaN,coupe,white,al


#### Get basic information about the dataset size and data types
 - We note that: 
    - There are 426880 rows and 18 columns
    - Year is a float.  We will conver this to integer
    - We have some null values that need further exploration

In [20]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

#### Quick glance at numeric values, min, max, means and standard deviation
 - We note some outliers for price and  odometer that we will need to explore further 

In [42]:
raw.describe()

,id,price,year,odometer
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07


#### Look for duplicate rows
  - No duplicate rows found

In [43]:
duplicate_rows=raw[raw.duplicated()]
duplicate_rows

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state


#### Inspect blanks, nulls, null ratios, duplicates
- From the below we can make the following observations:
    - Each car has a unique id so we can eliminate this column
    - We see a high number of VINs repeated throughout the dataset that we need to look at 
    - Due to the high ratio of nulls in several columns we have to decide whether to:
        - Drop the columns
        - Fill the cells with some sort of predictive value
        - Drop the rows containing these nulls 


In [35]:
def stats(df):
    unique_counts = []
    for col in df.columns:
        unique_values = df[col][~df[col].isna() & (df[col] != '') & (df[col] != ' ')].unique()
        unique_counts.append(len(unique_values))
    
    stats_df = pd.DataFrame({'blank_spaces': (df==' ').sum(),
                              'empty_strs': (df=='').sum(),
                              'nulls': df.isnull().sum(), 
                              'null_pct': round(100*(df.isnull().sum()) / len(df),2), 
                              'unique_not_null': df.nunique(),
                              'dups': [df[col].duplicated().sum() for col in df.columns]
                            }).sort_values(by='null_pct', ascending=False)
    return stats_df

stats(raw)

,blank_spaces,empty_strs,nulls,null_pct,unique_not_null,dups
size,0,0,306361,71.77,4,426875
cylinders,0,0,177678,41.62,8,426871
condition,0,0,174104,40.79,6,426873
VIN,0,0,161042,37.73,118246,308633
drive,0,0,130567,30.59,3,426876
paint_color,0,0,130203,30.50,12,426867
type,0,0,92858,21.75,13,426866
manufacturer,0,0,17646,4.13,42,426837
title_status,0,0,8242,1.93,6,426873
model,0,0,5277,1.24,29649,397230


##### 

In [41]:
value_counts = [raw[i].value_counts() for i in raw.columns]
value_counts

[7222695916    1
 7313139418    1
 7313423023    1
 7313423324    1
 7313424533    1
              ..
 7314655506    1
 7314655833    1
 7314657468    1
 7314659947    1
 7301591129    1
 Name: id, Length: 426880, dtype: int64,
 columbus                   3608
 jacksonville               3562
 spokane / coeur d'alene    2988
 eugene                     2985
 fresno / madera            2983
                            ... 
 meridian                     28
 southwest MS                 14
 kansas city                  11
 fort smith, AR                9
 west virginia (old)           8
 Name: region, Length: 404, dtype: int64,
 0        32895
 6995      3169
 7995      3129
 9995      2867
 8995      2837
          ...  
 21298        1
 49217        1
 63195        1
 19709        1
 17873        1
 Name: price, Length: 15655, dtype: int64,
 2017.0    36420
 2018.0    36369
 2015.0    31538
 2013.0    30794
 2016.0    30434
           ...  
 1943.0        1
 1915.0        1
 1902.0     

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [ ]:
#attribute selectoin
#data cleaning
#construction of new attributes
#transformations of data for modeling tools

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [ ]:
#A key objective is to determine if there is some important business issue that has not been sufficiently considered.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

In [ ]:
#as simple as generating a report or as complex as implementing a repeatable data mining process
#done by the user usually
#understand what actions are needed to make use of models
#living process/document